# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,broome,42.2506,-75.8330,5.67,76,20,3.09,US,1699919339
1,1,lillesand,58.2488,8.3778,1.05,80,100,11.32,NO,1699919340
2,2,dudinka,69.4058,86.1778,-16.32,88,69,4.75,RU,1699919341
3,3,edinburgh of the seven seas,-37.0676,-12.3116,13.64,77,7,11.14,SH,1699919128
4,4,west island,-12.1568,96.8225,25.99,83,75,4.12,CC,1699919045


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [29]:
%%capture --no-display

# Configure the map
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City"    
)

# Display the map plot
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [37]:
# Narrow down cities that fit criteria and drop any results with null values
filtered = city_data_df[(city_data_df["Max Temp"] < 20) & (city_data_df["Max Temp"] > 19)]

# Drop any rows with null values
filtered = filtered.dropna()

# Display sample data
filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
192,192,kot samaba,28.5522,70.4684,19.10,40,89,2.46,PK,1699919544
230,230,menongue,-14.6585,17.6910,19.24,69,81,1.74,AO,1699919584
263,263,kisesa,-3.0833,34.1500,19.50,80,77,1.94,TZ,1699919615
265,265,murzuq,25.9155,13.9184,19.32,51,0,4.76,LY,1699919616
340,340,puerto santa,-8.9906,-78.6456,19.80,85,89,4.93,PE,1699919715
354,354,camana,-16.6228,-72.7111,19.18,84,71,2.97,PE,1699919729
366,366,sibi,29.5448,67.8764,19.44,32,62,0.99,PK,1699919741
551,551,viedma,-40.8135,-62.9967,19.45,21,10,6.06,AR,1699919915
561,561,el grao,39.7980,-0.1321,19.08,75,16,1.81,ES,1699919945


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Create a copy of the original DataFrame with selected columns
hotel_df = filtered.copy()[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty "Hotel Name" column
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
192,kot samaba,PK,28.5522,70.4684,40,
230,menongue,AO,-14.6585,17.6910,69,
263,kisesa,TZ,-3.0833,34.1500,80,
265,murzuq,LY,25.9155,13.9184,51,
340,puerto santa,PE,-8.9906,-78.6456,85,
354,camana,PE,-16.6228,-72.7111,84,
366,sibi,PK,29.5448,67.8764,32,
551,viedma,AR,-40.8135,-62.9967,21,
561,el grao,ES,39.7980,-0.1321,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set the parameters for the type of search
limit = 1
radius = 10000
categories = "hotel"
conditions = "lodging"

# set up a parameters dictionary
params = {
    "radius":radius,
    "categories":categories,
    "conditions":conditions,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kot samaba - nearest hotel: No hotel found
menongue - nearest hotel: No hotel found
kisesa - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
puerto santa - nearest hotel: No hotel found
camana - nearest hotel: No hotel found
sibi - nearest hotel: No hotel found
viedma - nearest hotel: No hotel found
el grao - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
192,kot samaba,PK,28.5522,70.4684,40,No hotel found
230,menongue,AO,-14.6585,17.6910,69,No hotel found
263,kisesa,TZ,-3.0833,34.1500,80,No hotel found
265,murzuq,LY,25.9155,13.9184,51,No hotel found
340,puerto santa,PE,-8.9906,-78.6456,85,No hotel found
354,camana,PE,-16.6228,-72.7111,84,No hotel found
366,sibi,PK,29.5448,67.8764,32,No hotel found
551,viedma,AR,-40.8135,-62.9967,21,No hotel found
561,el grao,ES,39.7980,-0.1321,75,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
%%capture --no-display

# Configure the map
hover_cols = ["City", "Country", "Hotel Name"]

map_plot1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols=hover_cols    
)

# Display the map plot
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)